* ver.1:201903, ver.2:20200815, ver.3:20210902, ver.4:20221028

---
## 20221028 update : START

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
pip install jpype1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 453 kB 28.5 MB/s 


In [ ]:
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
import os
import pandas as pd

 
# txt파일을 넣을 폴더를 생성한다.(필요시 사용한다)
# song_folder_path = "/content/gdrive/My Drive/Colab Notebooks/song_folder_ver2"
# if not os.path.exists(song_folder_path):
#     os.makedirs(song_folder_path)

# 데이터프레임을 위한 리스트 생성(2022.10.28)
year_list=[]
rank_list = []       # 컬럼1 : 순위 리스트
title_list = []      # 컬럼2 : 곡명(텍스트) 리스트
songid_list = []     # 컬럼3 : 노래id 리스트
singer_list = []     # 컬럼4 : 가수명(텍스트) 리스트
singerid_list = []   # 컬럼5 : 가수id 리스트
# composer_list = []   # 컬럼6 : 작곡가(텍스트) 리스트
# lyricist_list = []   # 컬럼7 : 작사가(텍스트) 리스트
# lyricistid_list = [] # 컬럼8 : 작사가id 리스트
genre_list=[]         # 장르 리스트
lyrics_list = []     # 컬럼9 : 가사(텍스트) 리스트

In [ ]:
years=2015
for year in range(years,years+7) :     # 연도 범위 지정
    year=int(year)
    
    
    # OOOO년 노래 시대별차트
    # https://www.melon.com/chart/age/list.htm?idx=1&chartType=YE&chartGenre=POP&chartDate=2007&moved=Y
    headers = {
        'User-Agent': ('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
                       '(KHTML, like Gecko) Chrome/68.0.3440.75 Safari/537.36')
    }

    age_url = "https://www.melon.com/chart/age/list.htm"

    params = {
        'idx': '1',
        'chartType': 'YE',     # 10년 단위로 검색하는 부분과 연관
        'chartGenre': 'KPOP',  # 가요검색: KPOP, 팝송검색: POP
        'chartDate': str(year),   # 검색연도   ex) ('chartDate': '1970',)
        'moved': 'Y',
    }
    # 연도 하위 폴더 생성하기
    # os.mkdir('/content/gdrive/My Drive/Colab Notebooks/Lyrics_20210902/'+str(params['chartDate'])+'/')


    response = requests.get(age_url, params=params, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    song_list = soup.select('.lst50, .lst100')

    for i, meta in enumerate(song_list,1):
        year_list.append(str(year))  # 연도 리스트

        # 노래 id 추출 추가 2021.09.02 (출처 https://kin.naver.com/qna/detail.naver?d1id=1&dirId=10402&docId=398921342)
        for a in meta.select('a[href*=playSong]'):                         
          id1, id2 = re.findall(r'\'(\d+)\',\'(\d+)\'', a['href'])[0]        # id2가 노래id

        ### 순위, 곡명
        rank = i
        try:
            title = meta.select('a[href*=playSong]')[0].text
        except:
            title = meta.select('.wrap_song_info .ellipsis')[0].text
        title = title.strip()
        
        # print(str(params['chartDate']), "순위:",str(rank ), "곡명:",title, "노래id:",id2)
        rank_list.append(str(rank).strip())
        title_list.append(title)
        songid_list.append(id2)

        # 노래 데이터 url의 html 추출
        song_id_html = str(meta.select('a[onclick*=SongDetail]'))
        matched = re.search(r"\'(\d+)\'", song_id_html)                
        song_id = matched.group(1)                                    # song_id 추출 코드(원래 있던 코드, 2021.09.02.)
        front_url = 'https://www.melon.com/song/detail.htm?songId='
        song_url = front_url + song_id
        # print(song_id)

        ### 가수, 앨범명, 발매날짜, 장르
        response = requests.get(song_url, params=params, headers=headers)
        soup = BeautifulSoup(response.text, 'html.parser')
        singer_html = soup.select('.wrap_info .artist a')
        # print(singer_html)

        ### 가수
        singer_s = []
        if len(singer_html) != 0:
            for html in singer_html:
                singer_s.append(html['title'])
        else:
            # url 없는 Various Artists용
            singer_html = str(soup.select('.wrap_info .artist')[0])
            singer_html = singer_html.replace('\t','').replace('\r','').split('\n')
            singer_html = ''.join(singer_html)
            matched = re.search(r">(.*)<", singer_html)
            singer_s.append(matched.group(1))

        # 가수가 여러명일 때 하나의 string으로 표현
        singer_s = ', '.join(singer_s)

        # print(singer_s)
        singer_list.append(singer_s)

        #********************** 가수 id 추출 추가 2021.09.02 **********************#
        for c in soup.select('.wrap_info .artist a'):
          singer_id=re.findall(r'\'(\d+)\'', c['href'])[0]
        # print(rank, title, id2, singer_s, singer_id)
        singerid_list.append(singer_id) 

        # 작사가 id 찾아내기
        for d in soup.select('.section_prdcr .list_person .entry .artist'):
          prdcr_id=d.select('a')[0]['href']
          prdcr_id=re.findall('\d+', prdcr_id )
          # print(d.text.strip(), ''.join(prdcr_id))


        ### 작사가, 작곡가, 편곡가
        # 작사, 작곡, 편곡이 자리가 정해지지 않고, 존재하는 데이터만 앞쪽으로 밀려서 나온다.
        song_members = []
        for i in range(0,20):
            try:
                song_member = str(soup.select('.section_prdcr .list_person .entry .artist')[i].text).strip()
                d=soup.select('.section_prdcr .list_person .entry .artist')[i]
                prdcr_id=d.select('a')[0]['href']
                prdcr_id=re.findall('\d+', prdcr_id )
                song_member = song_member +'|'+ ''.join(prdcr_id)
                # print(song_member)
                member_data = str(soup.select('.section_prdcr .list_person .entry .meta')[i].text).strip()
                song_members.append([song_member, member_data])
            except:
                pass
        
        composer = ""
        lyricist = ""
        arranger = ""
        for data in song_members:
            if data[1] == '작사':
                lyricist = data[0]
            if data[1] == '작곡':
                composer = data[0]
            if data[1] == '편곡':
                arranger = data[0]     
        sms=str(song_members)
        sms=sms.replace("[[","[")
        # print(str(params['chartDate']),"|", rank,"|",title, "|작사수|",sms.count("작사"), "|작곡수|", sms.count("작곡"), "|편곡수|",sms.count("편곡"),song_members )
        # print(str(params['chartDate']),"|", rank, "|곡명|",title,"|",id2,"|가수|",singer_s, "|",singer_id,"|작사가|",lyricist,"|작곡가|", composer)
        # print("작곡가:", composer)
        # print("작사가:",lyricist)
        # print("편곡가:",arranger)
        # print("{0}-{1:03d} {2}({3})".format(year,rank,title, singer_s))

        # 장르
        song_genre_html = str(soup.select('.list dd')[2])
        matched = re.search(r">(.*)<", song_genre_html)
        song_genre = matched.group(1)
        genre_list.append(song_genre.strip())

        # 가사가 있으면 추출
        try:
            lyric_html = str(soup.select('.section_lyric .wrap_lyric .lyric')[0])
            lyric_html = lyric_html.replace('\t','').replace('\r','').split('\n')
            lyric_html = ''.join(lyric_html)

            matched = re.search(r"-->(.*)<br/>", lyric_html)
            lyric = matched.group(1).strip()
            lyric = lyric.replace('<br/>', '\n')

            #가사 앞뒤 빈칸 제거
            lyric_list = []
            for line in lyric.split('\n'):
                lyric_list.append(line.strip())
            lyric = ('\n').join(lyric_list)

        except:
            lyric = "없음"
        lyrics_list.append(lyric) 

info = {"year" : year_list, "rank" : rank_list, "title" : title_list, "song id" : songid_list ,"singer":singer_list,
        "singerid" : singerid_list, "genre" : genre_list, "lyrics": lyrics_list}
songs=pd.DataFrame(info)


In [ ]:
for value in info.values() :
  print(len(value), value)

700 ['2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2015', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '2016', '20

In [ ]:
import pandas as pd
songs=pd.DataFrame(info)
songs.head()

,year,rank,title,song id,singer,singerid,genre,lyrics
0,2015,1,뱅뱅뱅 (BANG BANG BANG),5719286,BIGBANG (빅뱅),198094,랩/힙합,난 깨어나 까만 밤과 함께\n다 들어와 담엔 누구 차례\n한 치 앞도 볼 수 없는 ...
1,2015,2,같은 시간 속의 너,5576417,나얼,3959,R&amp;B/Soul,꼭 그러지 않아도\n충분히 널 이해할 수 있어\n다른 사람 곁에 서 있는 니 모습이...
2,2015,3,꺼내 먹어요,5702706,Zion.T,537920,R&amp;B/Soul,안녕\n쉽지 않죠 바쁘죠\n왜 이렇게 까지\n해야 하나 싶죠\n바라는 게\n더럽게 ...
3,2015,4,이럴거면 그러지말지 (Feat. Young K),5704397,백아연,646173,발라드,궁금해서 잠이 안 와\n그때 왜 그랬어\n구차해도 묻고 싶어\n그때 난 뭐였어\n나...
4,2015,5,위잉위잉,5385974,혁오 (HYUKOH),785589,"인디음악, 록/메탈",비틀비틀 걸어가는 나의 다리\n오늘도 의미없는\n또 하루가 흘러가죠\n사랑도 끼리끼...


In [ ]:
songs.rename(columns={'singerid':'singer id'}, inplace=True)
songs.head()

,year,rank,title,song id,singer,singer id,genre,lyrics
0,2015,1,뱅뱅뱅 (BANG BANG BANG),5719286,BIGBANG (빅뱅),198094,랩/힙합,난 깨어나 까만 밤과 함께\n다 들어와 담엔 누구 차례\n한 치 앞도 볼 수 없는 ...
1,2015,2,같은 시간 속의 너,5576417,나얼,3959,R&amp;B/Soul,꼭 그러지 않아도\n충분히 널 이해할 수 있어\n다른 사람 곁에 서 있는 니 모습이...
2,2015,3,꺼내 먹어요,5702706,Zion.T,537920,R&amp;B/Soul,안녕\n쉽지 않죠 바쁘죠\n왜 이렇게 까지\n해야 하나 싶죠\n바라는 게\n더럽게 ...
3,2015,4,이럴거면 그러지말지 (Feat. Young K),5704397,백아연,646173,발라드,궁금해서 잠이 안 와\n그때 왜 그랬어\n구차해도 묻고 싶어\n그때 난 뭐였어\n나...
4,2015,5,위잉위잉,5385974,혁오 (HYUKOH),785589,"인디음악, 록/메탈",비틀비틀 걸어가는 나의 다리\n오늘도 의미없는\n또 하루가 흘러가죠\n사랑도 끼리끼...


In [ ]:
songs.to_excel("songs_raw.xlsx")

### <Reference>
* https://tinyurl.com/2y8zmznc
---
## 20221028 update : END
---